In [4]:
import glob
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize

In [114]:
# Need to download this if first time using nltk
# nltk.download('punkt')

In [5]:
DATA_PATH = '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/'

In [6]:
all_csv_paths=glob.glob(DATA_PATH+'*texts_and_info*')
all_csv_paths

['/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/nytimes_foxnews_article_texts_and_info.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/california_article_texts_and_info.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/texas_article_texts_and_info.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/illinois_article_texts_and_info.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/ohio_article_texts_and_info.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/florida_article_texts_and_info.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/newyork_article_texts_and_info.csv']

In [7]:
def load_and_convert(path):
    '''load one dataset from the path, drop the na, and convert date to timestamp'''
    df= pd.read_csv(path)
    # drop na of text field, right?
    df=df[df['text'].notna()]
    # convert to datetime so i can idmax
    return df.assign(publish_date=pd.to_datetime(df['publish_date']))

In [8]:
def drop_duplicates(df, columns=['url','title','text'], keep_col='publish_date', ignore_case=True):
    '''Check if either of the column in the input columns contains duplicates.
    If so, keep the row with the max value of the keep_col parameter. In our case, the latest timestamp.
    Because we are doing either, not all, I will need to run group by each column at a time untill exhaustion.
    '''

    if not ignore_case:
        for col in columns:
            df=df.loc[df.groupby(col)[keep_col].idxmax()]
    else:
        for col in columns:
            df=df.loc[df.groupby(df[col].str.lower())[keep_col].idxmax()]
 
    return df.sort_index()

In [119]:
# def get_sent_count(text):
#     '''return the sentence count of a text'''

#     return len(sent_tokenize(text))

In [10]:
for file_pat in all_csv_paths:
    temp_df= load_and_convert(file_pat)
    print(f" loading {file_pat} with {len(temp_df)} rows")
    temp_df_dedup=drop_duplicates(temp_df, ignore_case=True)
    print(f"de-duped! remaining rows {len(temp_df_dedup)}")

    temp_df_dedup['sent_count']=temp_df_dedup['text'].map(lambda text: len(sent_tokenize(text)))
    print(temp_df_dedup['sent_count'].describe())
    temp_df_dedup_drop_short_text=temp_df_dedup[temp_df_dedup['sent_count']>5]

    print(f"Final row counts: {len(temp_df_dedup_drop_short_text)}")

    new_path=file_pat.split('.csv')[0]+'_dedup.csv'
    # new_path=f"/home/crazybrokeasian/projects/{file_pat.split('/')[-1].split('.csv')[0]}_dedup.csv"
    print(f'saving new file to {new_path}')
    
    temp_df_dedup_drop_short_text.to_csv(new_path, index=False)

    print('--------')


 loading /home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/nytimes_foxnews_article_texts_and_info.csv with 3479 rows
de-duped! remaining rows 3213
count    3213.000000
mean       25.718332
std        38.975845
min         1.000000
25%        12.000000
50%        20.000000
75%        29.000000
max       787.000000
Name: sent_count, dtype: float64
Final row counts: 3066
saving new file to /home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/nytimes_foxnews_article_texts_and_info_dedup.csv
--------
 loading /home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/california_article_texts_and_info.csv with 14450 rows
de-duped! remaining rows 8476
count    8476.000000
mean       45.690066
std        55.563920
min         1.000000
25%        19.000000
50%        33.000000
75%        54.000000
max       834.000000
Name: sent_count, dtype: float64
Final row counts: 8059
saving new file to /home/pranavgoel/trans-fer-entropy/obtaining_news_collections/dat